In [1]:
# -*- coding: utf-8 -*-
import os, re, json, time
from typing import Dict, Any, List
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from docx import Document
from datetime import datetime

/home/alpaco/anaconda3/envs/seirah/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from sentence_transformers import SentenceTransformer

load_dotenv()  

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("API 키가 로드되었나요?", bool(os.getenv("OPENAI_API_KEY")))

EMBED_MODEL = "intfloat/e5-large"
GPT_MODEL = "gpt-4-turbo"
NLI_MODEL = 
embedder = SentenceTransformer(EMBED_MODEL)

SAVE_DIR = "./outputs"
os.makedirs(SAVE_DIR, exist_ok=True)

API 키가 로드되었나요? True


In [3]:
# 2️⃣  문서 검색 (E5 임베딩 기반)
# ────────────────────────────────────────────────────────────────
def search_reference(query: str, corpus: List[str], topk: int = 5):
    q_emb = embedder.encode([query], normalize_embeddings=True)
    c_emb = embedder.encode(corpus, normalize_embeddings=True)
    sims = cosine_similarity(q_emb, c_emb)[0]
    top_idx = np.argsort(sims)[::-1][:topk]
    return [(corpus[i], float(sims[i])) for i in top_idx]


[Hybrid Retrieval Node] → [Form Node] → [Context Node]
   ↓
[Draft Node (GPT 생성)] → [Validate Node] → [Export Node] → [RPA Node]

In [3]:
#  LLM + LangGraph Pipeline– Form → Context → Draft → Validate → Export
# 행정문서 자동 생성·검증, 민원분류, 정책문 초안 작성(GPT-계열 LLM)
# 3. GPT 기반 문서 생성
from typing import List, Tuple, Dict, Any
import os
import time
import json
import re


DOC_TEMPLATES = {
    # 1️ 정책 제안서 (AI 기반 행정혁신, 시정 개선안 등)
    "policy_proposal": {
        "system": (
            "당신은 지방자치단체의 정책 기획 전문가입니다. "
            "파주시의 데이터 기반 행정혁신 과제를 위한 정책 제안서를 작성하세요. "
            "모든 문장은 공식 보고서 문체로, 객관적 근거와 수치에 기반하여 작성합니다."
        ),
        "instructions": [
            "요약(Executive Summary)을 맨 앞에 작성",
            "정책 배경·현황 → 문제 정의 → 추진 필요성 → 제안 내용 → 기대효과 순서로 구성",
            "근거 데이터(통계청·경기도 데이터드림·삼성카드 등) 출처를 명시",
            "정책 추진 일정(연도별 단계)과 담당 부서를 제시",
            "예상 재정소요와 효과를 정량화(단위: 백만원, 인원, % 등)"
        ]
    },

    # 2️ 내부 보고서 (부서장 보고용)
    "internal_report": {
        "system": (
            "당신은 파주시청 내부 업무보고 문서를 작성하는 행정 전문가입니다. "
            "문서는 간결하되, 핵심 수치·추진 경과·차기 계획이 명확해야 합니다."
        ),
        "instructions": [
            "1페이지 이내 요약(핵심현황, 주요성과, 차기계획)을 먼저 제시",
            "사업명·담당부서·담당자·기간·예산을 명확히 표기",
            "성과와 문제점을 각각 3개 이내로 요약",
            "차기 추진계획은 구체적 일정과 액션 담당자 포함"
        ]
    },

    # 3️ 사업계획서 (공모·예산 신청용)
    "business_plan": {
        "system": (
            "당신은 지자체 사업계획서 작성 전문가입니다. "
            "파주시의 AI·데이터 기반 행정서비스 사업 제안서를 작성하세요."
        ),
        "instructions": [
            "기본 목차: 사업개요 → 추진배경 → 사업목표 → 세부 추진계획 → 예산계획 → 기대효과",
            "예산은 인건비·운영비·시스템 구축비 등 세부 항목으로 분류",
            "성과지표(KPI)는 정량적(예: 행정처리시간 20% 단축)으로 명시",
            "리스크 요인과 대응전략을 포함"
        ]
    },

    # 4️ 의회 보고서 / 주요 업무계획
    "assembly_brief": {
        "system": (
            "당신은 시의회 보고용 행정 현안 문서를 작성하는 전문가입니다. "
            "공식·격식 있는 문체로 주요 추진사항과 예산을 중심으로 보고서를 작성하세요."
        ),
        "instructions": [
            "개요 → 추진현황 → 문제점 및 개선계획 → 예산 및 일정 → 요청사항 순서로 구성",
            "핵심 내용은 불릿(•) 형태로 정리",
            "정책명·부서명·연도별 예산을 구체적으로 제시",
            "결론부에는 의회의 협조 요청 또는 승인 요청 문구 포함"
        ]
    },

    # 5️ 행정 공문 (기관 간 협조요청, 보고 등)
    "official_letter": {
        "system": (
            "당신은 파주시청의 공문 작성 전문가입니다. "
            "공문은 수신처·제목·본문·첨부·발신자(직위) 형식을 준수하며 "
            "공식·간결한 어조로 작성해야 합니다."
        ),
        "instructions": [
            "수신·참조·제목·본문·첨부·발신자 순서로 작성",
            "본문은 3단락 이내(요약, 협조요청내용, 추가문의)",
            "정책명·기간·협조범위는 구체적으로 명시"
        ]
    },

    # 6️ 기술보고서 (AI/데이터 분석 결과 보고)
    "technical_report": {
        "system": (
            "당신은 데이터 기반 행정분석 보고서를 작성하는 데이터사이언티스트입니다. "
            "파주시 관련 데이터를 분석한 결과를 객관적으로 정리하세요."
        ),
        "instructions": [
            "목차: 분석배경 → 데이터 개요 → 분석방법 → 주요결과(표/그래프 설명) → 시사점",
            "수치·모델명·데이터기간 등은 구체적으로 표기",
            "시사점은 정책 적용 가능성과 한계(리미트) 포함"
        ]
    },

    # 7️ 민원 요약 및 분류 보고서
    "civil_complaint_report": {
        "system": (
            "당신은 민원 데이터를 분석해 분류·요약 보고서를 작성하는 행정데이터 전문가입니다. "
            "AI 기반 민원 자동 분류 결과를 행정용 요약문으로 정리하세요."
        ),
        "instructions": [
            "민원 유형별(교통·환경·복지 등) 주요 빈도와 트렌드 제시",
            "지역별(읍면동 단위) 상위 3개 민원 이슈 표로 정리",
            "유형별 개선방안 제시(담당 부서/조치기한 포함)",
            "데이터 출처(민원시스템, 접수기간 등) 명기"
        ]
    },

    # 8️ 보도자료 / 시민 안내문
    "press_release": {
        "system": (
            "당신은 파주시 홍보담당관실의 보도자료 작성 전문가입니다. "
            "시민이 쉽게 이해할 수 있도록 명료한 문체로 작성하세요."
        ),
        "instructions": [
            "헤드라인·리드문(핵심 메시지) → 본문(내용·시행시기) → 문의처 순서",
            "전문용어 대신 쉬운 표현 사용",
            "핵심 통계(예: 참여인원, 지원금액)는 구체적으로 제시"
        ]
    },

    # 9 회의록 (내부 간담회·정책협의)
    "minutes": {
        "system": (
            "당신은 행정회의록을 작성하는 비서관입니다. "
            "모든 발언은 요약·객관적으로 기록하고, 결정사항과 담당자를 명확히 하세요."
        ),
        "instructions": [
            "회의명·일시·참석자·주요안건을 상단에 표기",
            "논의내용은 요약하되, 핵심결정 및 조치사항을 별도 항목으로 정리",
            "각 조치사항에는 담당부서·책임자·기한을 명시"
        ]
    }
}

def build_prompt(section_name: str, references: str, constraints: List[str], doc_type: str):
    base_rules = [
        "근거 불명 정보는 금지합니다. 출처를 명기하세요.",
        "수치·정책명·기간 등은 가능한 한 구체적으로 제시하세요.",
        "문장은 보고서 수준의 완결된 형태로 유지하세요."
    ]
    template = DOC_TEMPLATES.get(doc_type)
    if template:
        system_role = template["system"]
        doc_instructions = template["instructions"]
    else:
        system_role = "당신은 행정문서 작성 전문가입니다. 객관적이고 근거기반으로 작성하세요."
        doc_instructions = []

    all_constraints = base_rules + doc_instructions + constraints

    # 모델에게 구조화(JSON) 응답을 요청 (파싱 용이)
    user_prompt = f"""
# 작성항목: [{section_name}]
# 문서유형: [{doc_type}]
# 작성조건: {', '.join(all_constraints)}
# 참고자료:
{references}

다음 기준에 맞춰 내용을 생성하세요:
1) 출력 형식: **JSON**. 키는 아래와 같아야 합니다:
   {{
     "summary": "한 문단(요약)",
     "body": "상세 본문(여러 문단 가능)",
     "recommendations": [{{{{"title":"", "detail":"", "impact_estimate":"(예: 연간 비용/효과)"}}}}],
     "action_items": [{{{{"task":"", "owner":"", "due":"YYYY-MM-DD"}}}}],
     "references": ["출처1(기관, 연도)", "출처2(링크/문헌)"]
   }}
2) 각 주장 옆에 간단한 출처 주석을 괄호형태로 표기하세요 (예: (통계청, 2023)).
3) 표나 수치가 필요하면 간단한 표/항목 리스트로 제시하세요.
4) 문체: 한국어 보고서 문체(공식적).
"""

    return system_role, user_prompt

def _truncate_references(references: str, max_chars: int = 6000) -> str:
    if len(references) <= max_chars:
        return references
    # 안전하게 문장 단위로 자르기
    truncated = references[:max_chars]
    last_period = truncated.rfind("\n")
    if last_period > 0:
        truncated = truncated[:last_period]
    truncated += "\n\n[참고: 원문이 너무 김 - 일부만 제공됨]"
    return truncated

def _validate_output(parsed: Dict[str, Any]) -> Tuple[bool, List[str]]:
    errors = []
    required_keys = ["summary", "body", "recommendations", "references"]
    for k in required_keys:
        if k not in parsed:
            errors.append(f"Missing key: {k}")

    # 숫자 포맷 체크 예시: 권고안 impact_estimate
    if "recommendations" in parsed:
        for i, r in enumerate(parsed["recommendations"]):
            if "impact_estimate" in r and r["impact_estimate"]:
                # 간단 숫자 존재 확인 (숫자 또는 원화 표기)
                if not re.search(r"\d", str(r["impact_estimate"])):
                    errors.append(f"recommendations[{i}].impact_estimate에 숫자가 포함되어야 합니다.")

    # references 최소 1개 이상
    if "references" in parsed and len(parsed["references"]) == 0:
        errors.append("references는 하나 이상 필요합니다.")

    return (len(errors) == 0), errors

def gpt_generate(
    section_name: str,
    references: str,
    constraints: List[str],
    doc_type: str = "internal_report",
    temperature: float = None,
    max_tokens: int = 1200,
    retries: int = 3
) -> Dict[str, Any]:
    # doc_type별 권장 온도(있으면 사용)
    temp_map = {
        "policy_proposal": 0.2,
        "internal_report": 0.1,
        "official_letter": 0.0,
        "business_plan": 0.2,
        "technical_report": 0.1,
        "press_release": 0.3,
        "minutes": 0.0
    }
    if temperature is None:
        temperature = temp_map.get(doc_type, 0.2)

    # references 길이 조절
    safe_references = _truncate_references(references, max_chars=5000)

    system_role, user_prompt = build_prompt(section_name, safe_references, constraints, doc_type)

    attempt = 0
    last_exc = None
    while attempt < retries:
        try:
            res = client.chat.completions.create(
                model=GPT_MODEL,
                messages=[
                    {"role": "system", "content": system_role},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            text = res.choices[0].message.content.strip()

            try:
                parsed = json.loads(text)
            except json.JSONDecodeError:
                # 모델이 코드블록 등으로 감싸진 경우 정리
                # ```json\n{...}\n``` 제거 시도
                m = re.search(r"```(?:json)?\s*(\{.*\})\s*```", text, re.DOTALL)
                if m:
                    payload = m.group(1)
                    parsed = json.loads(payload)
                else:
                    # fallback: 텍스트를 body로 넣어 최소 출력 보장
                    parsed = {
                        "summary": "",
                        "body": text,
                        "recommendations": [],
                        "action_items": [],
                        "references": []
                    }

            # 기본 검증 수행
            ok, errors = _validate_output(parsed)
            if not ok:
                # 검증 실패 시 모델에 재요청(간단 리피드)
                # 여기서는 오류를 기록하고 재시도하도록 함
                attempt += 1
                last_exc = Exception(f"Output validation failed: {errors}")
                time.sleep(1 + attempt * 1.5)
                continue

            # 메타 추가
            parsed["_meta"] = {
                "model": GPT_MODEL,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "generated_at": time.strftime("%Y-%m-%dT%H:%M:%S%z")
            }
            return parsed

        except Exception as e:
            last_exc = e
            attempt += 1
            backoff = 1.5 ** attempt
            time.sleep(backoff)
            continue

    # 모든 시도 실패 시 예외 발생
    raise RuntimeError(f"gpt_generate failed after {retries} attempts. last error: {last_exc}")

In [4]:
import sys
sys.path.append("/home/alpaco/homework/paju_dacon")

In [8]:
from GPT_UNIEVAL_NLI_Pipeline import run_pipeline
run_pipeline(
    query="하이브리드 검색을 통해 행정문서 자동화를 위한 규정/서식 요건을 통합",
    section_name="도입 배경 및 필요성",
    doc_type="행정 자동화 기획서",
    constraints=["근거 기반", "중복 최소화", "정량적 수치 포함", "UNIEVAL 기준 준수"],
    corpus_dir="/home/alpaco/homework/paju_dacon/corpus",
    out_dir="/home/alpaco/homework/paju_dacon/outputs"
)

plt.show()  # Notebook에 그래프 즉시 표시

KeyboardInterrupt: 

In [ ]:
run_pipeline(
    query="하이브리드 검색을 통해 행정문서 자동화를 위한 규정/서식 요건을 통합",
    section_name="도입 배경 및 필요성",
    doc_type="행정 자동화 기획서",
    constraints=["근거 기반", "중복 최소화", "정량적 수치 포함", "UNIEVAL 기준 준수"],
    corpus_dir="/home/alpaco/homework/paju_dacon/corpus",
    out_dir="/home/alpaco/homework/paju_dacon/outputs"
)

plt.show()  # Notebook에 그래프 즉시 표시
return bar_path, radar_path  # 경로 반환

In [17]:
#  NLI 기반 Validator + UNIEVAL 평가 모듈
# 행정기록의 위변조 방지 및 결재 이력 추적( 전자문서 이력관리·결재 로그 보안)
# 4. GPT 기반 품질 평가
def gpt_validate(section_name: str, draft_text: str):
    prompt = f"""
당신은 행정 문서 품질평가 전문가입니다.
아래 작성문을 항목별로 평가해주세요.

[평가기준]
1. 정확성 (Accuracy) : 사실과 규정 근거의 일치 여부
2. 연관성 (Relevance) : 제목과 내용의 일치 정도
3. 일관성 (Consistency) : 수치·단위·논리의 균형
4. 유창성 (Fluency) : 문장 구조와 표현의 자연스러움
5. 중복도 (Redundancy↓) : 불필요 반복의 존재 여부

출력형식(JSON):
{{
  "Accuracy": 0~1,
  "Relevance": 0~1,
  "Consistency": 0~1,
  "Fluency": 0~1,
  "Redundancy": 0~1,
  "Comment": "개선 의견 요약"
}}

[평가대상: {section_name}]
{draft_text}
"""
    res = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "당신은 평가전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )
    try:
        data = json.loads(res.choices[0].message.content)
    except Exception:
        data = {"Accuracy": 0.7, "Relevance": 0.7, "Consistency": 0.7, "Fluency": 0.7, "Redundancy": 0.2, "Comment": "형식적 오류 없음"}
    return data


In [ ]:
# 자동 DOCX Export + 클라우드 저장
# (협업 행정 및 SaaS형 행정 서비스)	
# 문서 자동화 시스템을 클라우드형 워크스페이스로 확장
def export_report(sections: List[Dict[str, Any]], out_path: str):
    doc = Document()
    doc.add_heading("AI 기반 행정·R&D 문서 자동생성 보고서", level=0)
    doc.add_paragraph(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    for s in sections:
        doc.add_heading(s["title"], level=1)
        doc.add_paragraph(s["draft"])
        doc.add_heading("품질평가", level=2)
        doc.add_paragraph(json.dumps(s["eval"], ensure_ascii=False, indent=2))
    doc.save(out_path)
    print(f"✅ 보고서 저장 완료: {out_path}")

In [ ]:
# 6.  메인 파이프라인
def run_pipeline(sections_config: List[Dict[str, Any]], reference_corpus: List[str]):
    all_sections = []
    for sec in sections_config:
        title = sec["section"]
        refs = search_reference(title, reference_corpus, topk=3)
        ref_text = "\n\n".join([r[0] for r in refs])
        draft = gpt_generate(title, ref_text, sec.get("constraints", []))
        eval_res = gpt_validate(title, draft)
        all_sections.append({
            "title": title,
            "draft": draft,
            "eval": eval_res
        })
        print(f"[완료] {title} (평가점수: {eval_res})")

    out_path = os.path.join(SAVE_DIR, f"RND_Report_{datetime.now().strftime('%Y%m%d_%H%M')}.docx")
    export_report(all_sections, out_path)
    return out_path

### 스마트 행정 생태계는 다음과 같은 핵심 요소들로 구성됩니다.
•	첨단 기술 기반 시설: 센서, IoT 기기, 고속 통신망, 클라우드 시스템 등 기술적 기반이 필수적

•	데이터 통합 및 분석 플랫폼: 도시에서 발생하는 다양한 데이터를 수집, 통합, 분석하여 정책 수립과 의사결정에 활용하는 데이터 허브 역할

•	스마트 서비스: AI 챗봇을 활용한 24시간 민원 응대, 자율주행 셔틀, 스마트 환경 관리 솔루션 등 구체적인 시민 체감형 서비스가 포함

•	협력적 거버넌스: 민-관-학이 협력하여 도시 문제를 발굴하고 해결책을 모색하는 리빙랩(Living Lab) 방식의 운영 체계가 중요

•	인적 자산 및 제도: 변화에 대한 긍정적인 태도를 가진 행정 인력과 혁신을 지원하는 규제 개선(규제 샌드박스 등) 노력이 필요
 
### 기대 효과 및 변화
•	행정 효율성 증대: 반복적인 업무를 자동화하고(AI 활용 문서 요약 등), 신속한 행정 처리를 가능

•	서비스 품질 향상: 시민들에게 맞춤형 서비스를 제공하고(스마트 돌봄 등), 민원 처리 속도와 만족도 높임

•	선제적 문제 해결: 데이터를 기반으로 사회 문제를 예측하고 선제적으로 대응

  ➔ 재난 안전, 교통, 환경 등 다양한 분야의 도시 문제를 효과적으로 관리

•	지속가능한 도시 발전: 한정된 자원을 효율적으로 사용하여 에너지 절감 및 환경 보호에 기여하고, 지속가능한 도시 성장도모